In [ ]:
import pandas as pd
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

pd.options.plotting.backend = "plotly"

In [ ]:
cc = pd.read_csv('data\mabee2606_features_QF.csv', usecols=[0,1,7])
cc.rename(columns={'perfAngleFromHighSideClockwiseDegrees': 'AngleFromHighSideClockwiseDegrees'}, inplace=True)
cc['type'] = 'CC'
cc['NV'] = ['NV' if 'NV' in name else 'Inc' if 'Inc' in name else pd.NA for name in cc['name']]
cc.loc[(cc['NV'] == 'NV') | (cc['NV'] == 'Inc') , 'AngleFromHighSideClockwiseDegrees'] = pd.NA
cc['CC'] = ['CC'+name.split('.')[0] for name in cc['name']]


perfs = pd.read_csv('data\mabee2606_perfs_BIASED.csv', usecols=[0,1,7])
perfs['type'] = 'PERFS'
perfs.rename(columns={'perfAngleFromHighSideClockwiseDegrees': 'AngleFromHighSideClockwiseDegrees'}, inplace=True)

In [ ]:
features = cc[['depth', 'CC', 'AngleFromHighSideClockwiseDegrees']].groupby(['CC']).mean(numeric_only=False)
features['name'] = features.index

In [ ]:
features = cc[['depth', 'CC', 'AngleFromHighSideClockwiseDegrees']].groupby(['CC']).mean(numeric_only=False)
features['name'] = features.index
features = features.sort_values(by=['depth'], ascending=False)
features.reset_index(drop=True, inplace=True)
features['type'] = 'CC'
features.loc[features['AngleFromHighSideClockwiseDegrees'].isna(), 'type'] = features.loc[features['AngleFromHighSideClockwiseDegrees'].isna(), 'type']+' NV'
features.interpolate(method='pad', inplace=True)


features['fiberAngleFromHighSideClockwiseDegrees'] = [angle+180 if angle < 180 else angle-180 for angle in features['AngleFromHighSideClockwiseDegrees']]
features['fiber_plot_angle'] = [angle-360 if angle > 180 else angle for angle in features['fiberAngleFromHighSideClockwiseDegrees']]


combined = pd.concat([features, perfs])
combined['plot_angle'] = [angle-360 if angle > 180 else angle for angle in combined['AngleFromHighSideClockwiseDegrees']]
combined['angle_rounded'] = combined['AngleFromHighSideClockwiseDegrees'].round(0)
combined['fiber_angle_rounded'] = combined['fiberAngleFromHighSideClockwiseDegrees'].round(0)

In [ ]:
fig  = combined.plot(x='AngleFromHighSideClockwiseDegrees', y='depth', kind='scatter', color='type', custom_data=['name', 'angle'], color_discrete_map={'CC':'crimson', 'CC NV':'crimson', 'PERFS':'cornflowerblue'})
fig.update_layout(hovermode="y", title="Jewelry vs Perfs", legend_title="Type", legend_orientation="h", template="plotly")
fig.update_traces(hovertemplate='%{customdata[0]}<br>%{customdata[1]}')
fig.update_xaxes(range=[-185, 185])
fig.update_yaxes(autorange="reversed")

In [ ]:
fig = go.Figure()

# Add windows
gozone = [[combined.loc[combined['type']!='PERFS', 'plot_angle'].min(),combined.loc[combined['type']!='PERFS', 'plot_angle'].max()],
[combined.loc[combined['type']!='PERFS', 'depth'].min(), combined.loc[combined['type']!='PERFS', 'depth'].max()]]

nogo = [[combined.loc[combined['type']!='PERFS', 'fiber_plot_angle'].min(),combined.loc[combined['type']!='PERFS', 'fiber_plot_angle'].max()],
[combined.loc[combined['type']!='PERFS', 'depth'].min(), combined.loc[combined['type']!='PERFS', 'depth'].max()]]

fig.add_shape(type="rect",
    x0=gozone[0][0]-3, y0=gozone[1][0]-100, x1=gozone[0][1]+3, y1=gozone[1][1]+100,
    line=dict(width=0),
    fillcolor='rgba(154,205,50,0.1)', layer='below')

fig.add_shape(type="rect",
    x0=nogo[0][0]-3, y0=nogo[1][0]-100, x1=nogo[0][1]+3, y1=nogo[1][1]+100,
    line=dict(width=0),
    fillcolor='rgba(255,0,0,0.1)', layer='below')

# Add traces
fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='CC', 'plot_angle'], y=combined.loc[combined['type']=='CC', 'depth'], mode='markers', name='CC', marker_color='forestgreen', customdata=combined.loc[combined['type']=='CC', ['name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='CC NV', 'plot_angle'], y=combined.loc[combined['type']=='CC NV', 'depth'], mode='markers', name='CC NV', marker_color='forestgreen', customdata=combined.loc[combined['type']=='CC NV', ['name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='PERFS', 'plot_angle'], y=combined.loc[combined['type']=='PERFS', 'depth'], mode='markers', name='PERFS', marker_color='navy', customdata=combined.loc[combined['type']=='PERFS', ['name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined.loc[combined['type']!='PERFS', 'fiber_plot_angle'], y=combined.loc[combined['type']!='PERFS', 'depth'], mode='lines+markers', name='Fiber Wire', marker_color='crimson', customdata=combined.loc[combined['type']!='PERFS', ['name', 'fiber_angle_rounded']]))

fig.update_layout(hovermode="y", title="Jewelry vs Perfs", legend_title="Type", legend_orientation="h", template="plotly")
fig.update_traces(hovertemplate='%{customdata[0]}<br>%{customdata[1]}')
fig.update_xaxes(range=[-185, 185])
fig.update_yaxes(autorange="reversed")
fig.show()

In [ ]:
CC, CC_NV = features['type'].value_counts()

pie = make_subplots(rows=1, cols=1, specs=[[{"type": "domain"}]])

pie.add_trace(go.Pie(
     values=[CC, CC_NV],
     labels=['CC', 'No vertical component'],
     domain=dict(x=[0, 0.5]),
     name="Blast Protector",
     hoverinfo='label+value',
     sort=False,
     marker_colors = ['forestgreen', 'lightgrey']
     ), 
     row=1, col=1)

pie.update_layout(title_text="Success rate by jewelry type", template="plotly")
pie.show()

In [ ]:
pio.write_html(fig, file='plotly/m.html', auto_open=False)
pio.write_html(pie, file='plotly/mc.html', auto_open=False)

Well 2

In [ ]:
cc = pd.read_csv('data\Seidel Unit A3_features_QF.csv', usecols=[0,1,7])
cc.rename(columns={'perfAngleFromHighSideClockwiseDegrees': 'AngleFromHighSideClockwiseDegrees'}, inplace=True)
cc['type'] = ['CrossCouple' if 'CC' in name else 'BlastProtector' if 'BP' in name else 'Centralizer' for name in cc['name']]
cc['NV'] = ['NV' if 'NV' in name else pd.NA for name in cc['name']]
cc.loc[cc['NV'] == 'NV', 'AngleFromHighSideClockwiseDegrees'] = pd.NA
cc['J'] = [name.split()[0] if 'J' in name.split()[0] else pd.NA for name in cc['name']]
cc['BP'] = [name.split()[1][:3] if 'BP' in name.split()[1] else pd.NA for name in cc['name']]
cc['CC'] = [name.split()[1][:-2] if 'CC' in name.split()[1] else pd.NA for name in cc['name']]
cc['C'] = [name.split()[1] if name.split()[1].startswith('C1') else pd.NA for name in cc['name']]


perfs = pd.read_csv('data\Seidel Unit A3_Perfs_BIASED.csv', usecols=[0,1,7])
perfs['type'] = 'PERFS'
perfs.rename(columns={'perfAngleFromHighSideClockwiseDegrees': 'AngleFromHighSideClockwiseDegrees'}, inplace=True)

In [ ]:
features = pd.concat([cc[['depth', 'J', 'BP', 'AngleFromHighSideClockwiseDegrees']].groupby(['J','BP']).mean(numeric_only=False), cc[['depth', 'CC', 'AngleFromHighSideClockwiseDegrees']].groupby(['CC']).mean(numeric_only=False)])
features['name'] = features.index
features = features.sort_values(by=['depth'], ascending=False)
features.reset_index(drop=True, inplace=True)
features['type'] = ['BlastProtector' if 'BP' in name[1] else 'CrossCoupling' if 'CC' in name else 'Centralizer' for name in features['name']]
features.loc[features['AngleFromHighSideClockwiseDegrees'].isna(), 'type'] = features.loc[features['AngleFromHighSideClockwiseDegrees'].isna(), 'type']+' NV'
features.interpolate(method='pad', inplace=True)

features['fiberAngleFromHighSideClockwiseDegrees'] = [angle+180 if angle < 180 else angle-180 for angle in features['AngleFromHighSideClockwiseDegrees']]
features['fiber_plot_angle'] = [angle-360 if angle > 180 else angle for angle in features['fiberAngleFromHighSideClockwiseDegrees']]


combined = pd.concat([features, perfs])
combined['plot_angle'] = [angle-360 if angle > 180 else angle for angle in combined['AngleFromHighSideClockwiseDegrees']]
combined['angle_rounded'] = combined['AngleFromHighSideClockwiseDegrees'].round(0)
combined['fiber_angle_rounded'] = combined['fiberAngleFromHighSideClockwiseDegrees'].round(0)

In [ ]:
# fig  = combined.plot(x='plot_angle', y='depth', kind='scatter', color='type', custom_data=['name', 'angle'], color_discrete_map={'CrossCoupling':'lightsalmon', 'CrossCoupling NV':'lightsalmon', 'BlastProtector':'crimson', 'BlastProtector NV':'crimson', 'Centralizer':'lightblue', 'PERFS':'cornflowerblue'})
# fig.update_layout(hovermode="y", title="Jewelry vs Perfs", legend_title="Type", legend_orientation="h", template="plotly")
# fig.update_traces(hovertemplate='%{customdata[0]}<br>%{customdata[1]}')
# fig.update_xaxes(range=[-185, 185])
# fig.update_yaxes(autorange="reversed")

In [ ]:
fig = go.Figure()

p = ''.join([f'M {xy[0][0]+30},{xy[0][1]} ' if xy[1]==0 else f'S{xy[0][0]+30},{xy[0][1]} ' if xy[1]%2!=0 else f'{xy[0][0]+30},{xy[0][1]} ' for xy in zip(combined.loc[combined['type']!='PERFS', ['plot_angle','depth']].values,range(combined.loc[combined['type']!='PERFS', ['plot_angle','depth']].shape[0]))])

# Update axes properties
fig.update_xaxes(
    range=[-180, 180],
    zeroline=False,
)
fig.update_yaxes(
    range=[3500, 5000],
    zeroline=False,
)


poly = dict(type="path", path=p, fillcolor="PaleTurquoise", line_color="LightSeaGreen")

fig.update_layout(shapes=[poly])
fig.show()

In [ ]:
fig = go.Figure()

# Add windows
gozone = [[combined.loc[combined['type']!='PERFS', 'plot_angle'].min(),combined.loc[combined['type']!='PERFS', 'plot_angle'].max()],
[combined.loc[combined['type']!='PERFS', 'depth'].min(), combined.loc[combined['type']!='PERFS', 'depth'].max()]]

nogo = [[combined.loc[combined['type']!='PERFS', 'fiber_plot_angle'].min(),combined.loc[combined['type']!='PERFS', 'fiber_plot_angle'].max()],
[combined.loc[combined['type']!='PERFS', 'depth'].min(), combined.loc[combined['type']!='PERFS', 'depth'].max()]]

poly = dict(type="path", path=p, fillcolor='rgba(154,205,50,0.1)', line_color="LightSeaGreen")

fig.update_layout(shapes=[poly])

fig.add_shape(type="rect",
    x0=gozone[0][0]-3, y0=gozone[1][0]-100, x1=gozone[0][1]+3, y1=gozone[1][1]+100,
    line=dict(width=0),
    fillcolor='rgba(154,205,50,0.1)', layer='below')

fig.add_shape(type="rect",
    x0=nogo[0][0]-3, y0=nogo[1][0]-100, x1=nogo[0][1]+3, y1=nogo[1][1]+100,
    line=dict(width=0),
    fillcolor='rgba(255,0,0,0.1)', layer='below')

# Add traces
fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='BlastProtector', 'plot_angle'], y=combined.loc[combined['type']=='BlastProtector', 'depth'], mode='markers', name='BlastProtector', marker_color='forestgreen', customdata=combined.loc[combined['type']=='BlastProtector', ['name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='BlastProtector NV', 'plot_angle'], y=combined.loc[combined['type']=='BlastProtector NV', 'depth'], mode='markers', name='BlastProtector NV', marker_color='forestgreen', customdata=combined.loc[combined['type']=='BlastProtector NV', ['name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='CrossCoupling', 'plot_angle'], y=combined.loc[combined['type']=='CrossCoupling', 'depth'], mode='markers', name='CrossCoupling', marker_color='darkolivegreen', customdata=combined.loc[combined['type']=='CrossCoupling', ['name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='CrossCoupling NV', 'plot_angle'], y=combined.loc[combined['type']=='CrossCoupling NV', 'depth'], mode='markers', name='CrossCoupling NV', marker_color='darkolivegreen', customdata=combined.loc[combined['type']=='CrossCoupling NV', ['name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='PERFS', 'plot_angle'], y=combined.loc[combined['type']=='PERFS', 'depth'], mode='markers', name='PERFS', marker_color='navy', customdata=combined.loc[combined['type']=='PERFS', ['name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined.loc[combined['type']!='PERFS', 'fiber_plot_angle'], y=combined.loc[combined['type']!='PERFS', 'depth'], mode='lines+markers', name='Fiber Wire', marker_color='crimson', customdata=combined.loc[combined['type']!='PERFS', ['name', 'fiber_angle_rounded']]))

fig.update_layout(hovermode="y", title="Jewelry vs Perfs", legend_title="Type", legend_orientation="h", template="plotly", yaxis_title="Depth", xaxis_title='AngleFromHighSideClockwiseDegrees')
fig.update_traces(hovertemplate='%{customdata[0]}<br>%{customdata[1]}')
fig.update_xaxes(range=[-185, 185])
fig.update_yaxes(autorange="reversed")
fig.show()

In [ ]:
features['type'].value_counts()

In [ ]:
BP, CC, BP_NV, CC_NV = features['type'].value_counts()

pie = make_subplots(rows=1, cols=2, specs=[[{"type": "domain"}, {"type": "domain"}]])

pie.add_trace(go.Pie(
     values=[BP, BP_NV],
     labels=['BlastProtector', 'No vertical component'],
     domain=dict(x=[0, 0.5]),
     name="Blast Protector",
     hoverinfo='label+value',
     sort=False,
     marker_colors = ['forestgreen', 'lightgrey']
     ), 
     row=1, col=1)


pie.add_trace(go.Pie(
     values=[CC, CC_NV],
     labels=['CrossCoupling', 'No vertical component'],
     domain=dict(x=[0, 0.5]),
     name="Cross Coupling",
     hoverinfo='label+value',
     sort=False,
     marker_colors = ['darkolivegreen', 'lightgrey']
     ), 
     row=1, col=2)

pie.update_layout(title_text="Success rate by jewelry type", template="plotly")
pie.show()

In [ ]:
pio.write_html(fig, file='plotly/s.html', auto_open=False)
pio.write_html(pie, file='plotly/sc.html', auto_open=False)